In [3]:
import pandas as pd
import os

# 환경 변수 설정
os.environ['MECAB_PATH'] = '/usr/local/bin/mecab'
os.environ['MECAB_LIB_PATH'] = '/usr/local/lib'
os.environ['MECAB_DIC_PATH'] = '/usr/local/lib/mecab/dic/mecab-ko-dic'


from dotenv import load_dotenv
# .env 파일 경로 지정
dotenv_path = os.path.join("../", '.env')
# .env 파일 로드
load_dotenv(dotenv_path)
import pymysql
from tqdm import tqdm
import random
random.seed(1004)
import json
from konlpy.tag import Mecab, Okt
okt = Okt()
# Mecab 형태소 분석기 초기화
# mecab = Mecab(dicpath="/home/dothis/mecab/mecab-ko-dic-2.1.1-20180720")
# mecab = Mecab(dicpath="/usr/local/lib/mecab/dic/mecab-ko-dic")

from transformers import DebertaV2TokenizerFast

# 기존 모델의 토크나이저를 로드합니다.
tokenizer = DebertaV2TokenizerFast.from_pretrained('lighthouse/mdeberta-v3-base-kor-further')

/opt/anaconda3/envs/ai/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [12]:
# 샘플 텍스트와 NER 태그
texts = ["(여자)아이들은 매우 인기 있는 아이돌 그룹입니다.", 
         "나는 (여자)아이들의 신곡을 좋아합니다.", 
         "그녀는 (여자)아이들의 콘서트를 보러 갔습니다."]

# 엔티티 정보: [시작 인덱스, 끝 인덱스, 태그]
ner_entities = [[[0, 6, 'group']], 
                [[3, 9, 'group']], 
                [[4, 10, 'group']]]

# 데이터셋을 저장할 리스트
dataset = []

for text, entities in zip(texts, ner_entities):
    # 텍스트를 토큰화합니다.
    tokenized_text = tokenizer.tokenize(text)
    
    # NER 태그의 시작 및 끝 인덱스를 저장할 리스트
    ner_tags = []
    
    # 엔티티에 대한 NER 태그 생성
    for entity in entities:
        char_start_idx, char_end_idx, tag = entity
        # 텍스트의 시작과 끝 인덱스를 기반으로 토큰화된 텍스트에서 인덱스를 계산
        token_start_idx = len(tokenizer.tokenize(text[:char_start_idx]))
        token_end_idx = len(tokenizer.tokenize(text[:char_end_idx])) - 1
        
        ner_tags.append([token_start_idx, token_end_idx, tag])
    
    # 데이터셋에 추가
    dataset.append({
        'tokenized_text': tokenized_text,
        'ner': ner_tags
    })
    

# JSON 파일로 저장
with open('train_data.json', 'w', encoding='utf-8') as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [2]:

#### PROXMOX 데이터베이스 연결 정보
host = os.environ.get('PROXMOX_MYSQL_HOST') # RDS 엔드포인트 URL 또는 IP 주소
port = int(os.environ.get('PROXMOX_MYSQL_PORT')) # RDS 데이터베이스 포트 (기본값: 3306)
user = os.environ.get('PROXMOX_MYSQL_USER') # MySQL 계정 아이디
password = os.environ.get('PROXMOX_MYSQL_PW') # MySQL 계정 비밀번호
db_name = "dothis_svc" # 데이터베이스 이름

# PROXMOX MySQL 연결하기
proxmox_conn = pymysql.connect(host=host, 
                        port=port, 
                        user=user, 
                        password=password, 
                        db=db_name, 
                        charset='utf8mb4')

# 데이터베이스 커서(Cursor) 객체 생성
proxmox_cursor = proxmox_conn.cursor()
####

In [3]:
#### 테이블 가져오기
def collect_video_table_name():
    
    query = f"""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = '{db_name}'  
                AND table_name LIKE 'video_data_%';
            """
    proxmox_cursor.execute(query)
    data_table_names = [row[0] for row in proxmox_cursor.fetchall() if row[0] not in ["video_data_NULL", "video_data_temp", "video_data_9999"]]
    
    return data_table_names
####

tables = collect_video_table_name()

In [4]:
keyword = "먹방"
df = pd.DataFrame()
for table in tqdm(tables):
    query = f"select video_id, channel_id, video_title, video_published, video_cluster from {table} where video_title like '%{keyword}%' and \
            video_published > '2024-06-12';"
    proxmox_cursor.execute(query)
    etc = pd.DataFrame(proxmox_cursor.fetchall(), columns=["video_id", "channel_id", "video_title", "video_published", "video_cluster"])
    df = pd.concat([df, etc], axis=0)
df.video_published = pd.to_datetime(df.video_published)
df.reset_index(drop=True, inplace=True)
df


100%|██████████| 101/101 [02:33<00:00,  1.52s/it]


,video_id,channel_id,video_title,video_published,video_cluster
0,3EtVpZeinj0,UCgSkIO-lBMCUhrz7oDzYHlg,푸바오일기 3일차 누워서 양손 대나무 먹방,2024-06-14,0
1,4sanyH33bcc,UCLSAkTrvVpIFM4Eot7y67ng,참외먹방????키위먹방????참외10개먹기~,2024-06-14,0
2,927TRwQ1Y5o,UCXaP-UftW3J_Q9rjp6SGZng,Mukbang কোরিয়ানদের বাংলাদেশের পরোটা পছন্দ সাথ...,2024-06-14,0
3,B1eIjxi_9Gs,UCrmwZVtCchAXnGQeRtejDHg,경악스런 색깔. 민트초코라도 넣었냐? 호랑꾼 오션카레 #호랑꾼 #food #먹방 #...,2024-06-14,0
4,bOE_Gg-4Cg4,UCwMFK-rUdzOUM6QxIgzWcPQ,푸바오 중국 야외방사장 먹방/장하다 푸공주/씩씩한여장군 푸바오/멘탈 짱ㅋㅋ #슈푸스...,2024-06-14,0
...,...,...,...,...,...
2130,croLgwRlQxE,UC51f_RyApFUWLcptORQhdKg,변우석 싱가포르 팬미팅 전 라이브 방송/ 먹방,2024-06-30,93
2131,eWNi_VN_srE,UCQlm9fWbv3MkmfQLGqwgKgg,유신쇼 먹쇼먹방 시골청국장 김치전 행복한밥상 신청자 철이님 [20240630],2024-06-30,93
2132,Ik3T7eT1JEg,UCA_7R_B4OoG4idEfwurugbA,분모자 떡볶이 먹방,2024-06-30,93
2133,rJ9v2YWkFho,UCv6fLUAWutHkAhiP_xPtvlw,예당장어정식 먹방 6.30.2024,2024-06-30,93


In [5]:
sample = df.sample(10)
sample

,video_id,channel_id,video_title,video_published,video_cluster
950,KWROEzmbhQY,UCSRq0Th9o8t80LYS2iJ2-yA,"예니네 집밥 》 호박전 , 된장찌개 , 소세지전 가족먹방",2024-06-21,61
697,nt-WaJEgeZM,UCh08-7H91ss7mGXPcC7jPsA,제22회 퇴촌 토마토 축제 탐방(퇴촌 방울 토마토 먹방????????✨),2024-06-21,60
1155,cwEAXTHn4Aw,UC2_hzdvP-j569djTquAed_Q,"맛있는먹방♡ 도가니탕, 실비김치, 포기김치, 총각김치, 깍두기, 우리 입맛에는 김치...",2024-06-27,61
2088,S08540E3_Jc,UC7JB5aaqjzsCRK5Pfw6Yx3A,Animation Mukbang ASMR Compilation 애니 먹방 모음집 /...,2024-06-21,93
1207,slxZdmL1oj8,UC2g_1sZQw_qin0nOvc4kFSA,뼈해장국에 김치와 소주 먹방 ASMR MUKBANG | Pork Rib Hango...,2024-06-28,61
180,dx8UZHO0Sco,UCO7E5G042JFuhOX4eIUZPfw,오메킴 몰래 찾아와서 갑자기 삼겹살 먹방 하는 멤버들..(24.06.15#3),2024-06-15,10
492,nz-HR_mKoTQ,UCbbiEt2oAN3F5tJwAHOIPbA,[ASMR] ‎@AsmrWD 신기한물먹방 #mukbang #ASMRDRINKING...,2024-06-25,38
681,0acjqsMXhA8,UC4cfOoZIjpOJOwVG6bbioBw,지금 바로 시작합니다 푸바오 먹방 ASMR~빨리 들어와서 보자구요~,2024-06-21,59
939,cKjJs1E_jak,UC8ETQdBrlsTEAUIC5xCihsQ,"인프제(INFJ) 의 아빠표 간장게장????, 새우???? 먹방 리얼사운드 ASMR...",2024-06-21,61
1363,iRVsSHvinkk,UCfpaSruWW3S4dibonKXENjA,1인분에 2000원?! 사장님이 음식 맛없대요???? 과연.. 천호동 2천원 즉석떡...,2024-06-18,62


In [37]:
path = "./make_data.json"
# JSON 데이터를 UTF-8로 인코딩하여 파일로 저장
if os.path.exists(path):
    # JSON 파일 불러오기
    with open(path, 'r', encoding="utf-8-sig") as file:
        data = json.load(file)
    # Use a set to track seen origins
    seen_origins = set()
    unique_data = []

    # Filter out duplicates
    for entry in data:
        if entry['origin'] not in seen_origins:
            unique_data.append(entry)
            seen_origins.add(entry['origin'])
    data = unique_data
else:
    data = list()
len(data)

25

In [35]:
origin = sample.video_title.iloc[9]
tokenized_text = mecab.morphs(origin)
print([(i, n) for i, n in enumerate(tokenized_text)])
ner = [[20, 21, "Money"], [22, 23, "Food"], [24, 24, "media content"]]
_data = {"origin":origin, "tokenized_text":tokenized_text, "ner":ner}
data.append(_data)

[(0, '1'), (1, '인분'), (2, '에'), (3, '2000'), (4, '원'), (5, '?'), (6, '!'), (7, '사장'), (8, '님'), (9, '이'), (10, '음식'), (11, '맛없'), (12, '대요'), (13, '?'), (14, '???'), (15, '과연'), (16, '.'), (17, '.'), (18, '천호동'), (19, '2'), (20, '천'), (21, '원'), (22, '즉석'), (23, '떡볶이'), (24, '먹방')]


In [36]:
path = "./make_data.json"
# JSON 데이터를 UTF-8로 인코딩하여 파일로 저장
with open(path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)